## Generic programming 

One of Julia's strongest features is its **generic programming** ability, i.e. the possibility of writing an algorithm at a high level that can run efficiently in many different contexts, abstracting away many of the implementation and even hardware details.

In this lecture and the following one, we will explore the **power method** as a test case.

### The power method

The power method is a simple algorithm (not recommended for serious use!) to calculate the largest eigenvalue of a matrix.

Recall that $(\mathbf{v}, \lambda)$ is an eigenvector / eigenvalue pair for $\mathsf{A}$ if

$$\mathsf{A} \cdot \mathbf{v} = \lambda \mathbf{v},$$

where $\lambda \in \mathbb{C}$ is a scalar, which, in general, may be a complex number.

The algorithm is as follows:

> Take an arbitrary non-zero initial vector $\mathbf{v}_0$ and apply $\mathsf{A}$ to it many times, to obtain the sequence

> $$\mathbf{v}_i := \mathsf{A}^i \cdot \mathbf{v}_0.$$

This sequence of vectors converges to the eigenvector of $\mathsf{A}$ with the largest eigenvalue.

If the $\mathbf{v}_i$ are normalised (which, in practice, they usually must be so that they do not explode during the algorithm), then the eigenvalue is given by the limit of $\mathbf{v}_n \cdot \mathsf{A} \cdot \mathbf{v}_n$. In practice, with finite precision (e.g. standard `Float64`), this limit is attained after a finite number of steps.

### Implementation in Julia

Take, for example, the matrix

In [5]:
M = [2. 1; 1 1]

2×2 Array{Float64,2}:
 2.0  1.0
 1.0  1.0

and an initial non-zero, normalized vector:

In [11]:
v = rand(2)
normalize!(v)    # only in Julia v0.5
v /= norm(v)     # alternative: v = v / norm(v)

2-element Array{Float64,1}:
 0.865137
 0.501536

The basic step of the algorithm that must be repeated (use `Ctrl-Enter` in the notebook to repeatedly execute the same cell) is:

In [24]:
v = M * v
v /= norm(v)   #  or normalize!(v) in Julia v0.5

2-element Array{Float64,1}:
 0.850651
 0.525731

To check that this is indeed an eigenvector, we do an element-wise division:

In [25]:
(M*v) ./ v

2-element Array{Float64,1}:
 2.61803
 2.61803

### Power method function

Both for ease of use and efficiency, we must now wrap this in a function:

In [7]:
function power_method(M, v)
    for i in 1:100
        v = M*v        # repeatedly creates a new vector and destroys the old v
        v /= norm(v)
    end
    
    return v, norm(M*v) / norm(v)  # or  (M*v) ./ v
end

power_method (generic function with 1 method)

We could write a version that *modifies* its argument `v`, if we so desired (although in general this is usually a bad idea) with:

In [71]:
function power_method!(M, v)
    for i in 1:100
        v[:] = normalize(M*v)   # assign in-place to the elements of v
    end
    
    return v, norm(M*v) / norm(v)
end

power_method! (generic function with 1 method)

We can now apply our function:

In [27]:
power_method(M, rand(2))

([0.850651,0.525731],2.618033988749895)

**Exercise**: Modify the function so that it returns once the eigenvalue has converged to the eigenvalue, rather than doing a fixed number of iterations. [Check convergence of the eigenvalue, not the eigenvector.]

## What is generic about this?

This code is **generic**: we have not specified what kind of object `M` and `v` are, i.e. what **type** of object they are. We can now try using different types of arguments:

In [28]:
power_method(3, 1)

(1.0,3.0)

Here, we have effectively treated the number $3$ as an operator that takes $x$ and returns $3x$.  The power method is thus a complicated way to solve the equation $3x = \lambda x$ and show that the operator $3$ has eigenvalue $3$ and "eigenvector" $1$.

We could try to call the function e.g. with strings, but several things will go wrong, e.g. division of one string by another.

### Julia matrix types

Julia has a wide variety of predefined matrix types. For example:

In [29]:
D = Diagonal([1, 2, 3.])

3×3 Diagonal{Float64}:
 1.0   ⋅    ⋅ 
  ⋅   2.0   ⋅ 
  ⋅    ⋅   3.0

This is an efficient way to store a diagonal matrix: only the diagonal elements are actually stored, in `D.diag`:

In [37]:
D.diag

3-element Array{Float64,1}:
 1.0
 2.0
 3.0

The `Diagonal` type has sensible built-in behaviour; for example, we cannot set off-diagonal elements (which must be 0):

In [38]:
D[1,2] = 10

LoadError: LoadError: ArgumentError: cannot set an off-diagonal index (1, 2) to a nonzero value (10)
while loading In[38], in expression starting on line 1

but we can modify diagonal elements:

In [39]:
D[2,2] = 10

10

In [40]:
D

3×3 Diagonal{Float64}:
 1.0    ⋅    ⋅ 
  ⋅   10.0   ⋅ 
  ⋅     ⋅   3.0

Multiplication of a `Diagonal` matrix by a (dense) vector is defined:

In [49]:
v = [1, 2, 3.]
D*v

3-element Array{Float64,1}:
  1.0
 20.0
  9.0

We can check which method is actually being called here:

In [43]:
@which D*v

*(D::Diagonal, V::AbstractArray{T<:Any,1}) at linalg/diagonal.jl:117

By clicking through to the source code, we see that the definition of this method is very simple. 

We can create a dense (standard Julia matrix) version of `D`:

In [48]:
full(D)  # actual dense Julia matrix

3×3 Array{Float64,2}:
 1.0   0.0  0.0
 0.0  10.0  0.0
 0.0   0.0  3.0

and can write a test to see if the two ways of multiplying agree; make sure to always do tests like this when you define your own types:

In [51]:
using Base.Test
@test D*v == full(D)*v   # use ≈ (type as \approx<TAB>) for approximate equality 

Test Passed
  Expression: D * v == full(D) * v
   Evaluated: [1.0,20.0,9.0] == [1.0,20.0,9.0]

Since `*` is defined for `D` and vectors `v`, we can now just apply our `power_method`:

In [52]:
power_method(D, [1,1.,0.])

([1.0e-100,1.0,0.0],10.0)

Another predefined Julia matrix type is `SymTriDiagonal`, which is an efficient representation for symmetric, tridiagonal matrices, storing only the diagonal and super-diagonal elements:

In [53]:
n = 100
S = SymTridiagonal(rand(n), rand(n-1))
S[1:5, 1:5]

5×5 Array{Float64,2}:
 0.515808  0.654545   0.0       0.0       0.0     
 0.654545  0.0894668  0.241253  0.0       0.0     
 0.0       0.241253   0.845744  0.344888  0.0     
 0.0       0.0        0.344888  0.278708  0.633065
 0.0       0.0        0.0       0.633065  0.680082

Again, since matrix-vector multiplication with `*` is already defined, we can immediately apply our function:

In [58]:
vec, val = power_method(S, rand(n))

([2.21216e-14,4.78634e-14,3.05722e-13,9.31137e-13,2.26963e-12,1.07362e-10,3.6826e-9,1.47815e-8,7.7924e-8,1.54516e-7,2.75612e-7,7.0891e-7,8.2276e-7,2.60173e-6,1.93666e-5,2.95875e-5,4.809e-5,4.74514e-5,3.53911e-5,2.00268e-5,6.00597e-5,0.00030493,0.000521108,0.00110748,0.000994101,0.000761609,0.000152331,1.79893e-6,4.24719e-7,7.62555e-8,5.23806e-9,2.22988e-9,7.01621e-11,6.51593e-11,9.06091e-11,8.85583e-10,1.12622e-8,8.97324e-8,5.54233e-7,8.14079e-6,1.46452e-5,2.25342e-5,5.98377e-5,0.00161887,0.00636336,0.040697,0.0764824,0.147861,0.223097,0.299206,0.36192,0.272819,0.0126854,0.00626602,0.00123697,0.000590605,0.000167305,0.0001225,7.51248e-5,3.77119e-5,1.51731e-5,8.40828e-6,8.94393e-6,0.000218918,0.000449237,0.000367359,0.000276189,0.000111725,7.70659e-5,6.36498e-5,3.8899e-5,2.53071e-5,3.22871e-6,1.06037e-6,6.922e-7,3.23983e-7,5.20562e-6,1.04361e-5,1.42798e-5,1.73998e-5,7.66158e-5,0.0100365,0.022488,0.0603466,0.213355,0.363251,0.548386,0.368273,0.0833845,0.0227317,0.0100111,0.00534002,6.525

Let's compare the result with the built-in Julia function `eigvals`, which, in this case, uses a highly optimized function from the LAPACK library:

In [59]:
λ = eigvals(S)
val - maximum(abs(λ))

-0.006400446401666482

Indeed we found a good approximation to the top eigenvalue; a better approximation would require to iterate longer until convergence.

We may want to restrict the allowed types that we pass in to the function, e.g. by using the type signature

    function power_method(M, v::AbstractVector)

so that we can only pass objects of type `AbstractVector` as the second argument. On the other hand, we may lose some genericity like this.

### BigFloats

Julia includes a library for arbitrary-precision calculations, `BigFloat`.

In [1]:
M = BigFloat[2 1; 1 1.]

2×2 Array{BigFloat,2}:
 2.000000000000000000000000000000000000000000000000000000000000000000000000000000  1.000000000000000000000000000000000000000000000000000000000000000000000000000000
 1.000000000000000000000000000000000000000000000000000000000000000000000000000000  1.000000000000000000000000000000000000000000000000000000000000000000000000000000

The default is 256 binary digits (bits) of precision:

In [2]:
precision(M[1,1]) 

256

which is about 77 decimal digits:

In [4]:
log10(2)*precision(M[1,1])

77.06367888997919

This can be changed with

In [3]:
setprecision(BigFloat, 128)

128

We can now find an eigenvalue to high precision using the *same* algorithm:

In [5]:
v = [big(1.0), big(1.0)]

2-element Array{BigFloat,1}:
 1.000000000000000000000000000000000000000
 1.000000000000000000000000000000000000000

In [8]:
power_method(M, v)

(BigFloat[8.506508083520399321815404970630110722395e-01,5.257311121191336060256690848478766072835e-01],2.61803398874989484820458683436563811772)

For this particular matrix, an analytical expression for the eigenvalue may be obtained in the standard way, as a root of the characteristic polynomial $(\lambda - 2)(\lambda - 1) - 1 = 0$, giving

In [9]:
λ = (3 + sqrt(big(5))) / 2

2.61803398874989484820458683436563811772

In [10]:
vec, val = power_method(M, v)

(BigFloat[8.506508083520399321815404970630110722395e-01,5.257311121191336060256690848478766072835e-01],2.61803398874989484820458683436563811772)

In [12]:
λ - val

0.000000000000000000000000000000000000000